# ***MODEL 2*** : Demand Based Dynamic Pricing Model





## Preprocessing Of the Dataset

### **Importing the Libraries**:




In [25]:
!pip install pathway bokeh  --quiet

In [26]:
import pandas as pd
import pathway as pw
import numpy as np
import bokeh.plotting
import bokeh.model
import panel as pn
import datetime
from datetime import datetime

### Feature Engineering :  

In [27]:
def traffic_level(traffic):   # Converting the Categorical Columns to Numerically Contribute for Demand
  if traffic == 'low':
    return 3.0
  elif traffic == 'avg':
    return 6.0
  else:
    return 9.0

def vehicle_type_weight(type):  #Making Weightage of Each Vehicle
  if type == 'cycle':
    return 0.65
  elif type == 'bike':
    return .85
  elif type == 'car':
    return 1.15
  else:
    return 1.35


## Pipeline Creation

In [28]:
#Opening the CSV files , Add the Required Features , and Saved that files
files ={}
for i in range(1,15):

 file = f'f{i}'
 files[file] = pd.read_csv(f"/content/parking_lot_{i}.csv")  # reading the Files( These are the Files that was made with Model 1 Notebook)
 files[file]['vehicle_type_weight'] = files[file]['VehicleType'].apply(vehicle_type_weight)   # Making the Features
 files[file]['traffic_level'] = files[file]['TrafficConditionNearby'].apply(traffic_level)
 files[file].to_csv(f"/content/parking_lot_{i}.csv", index=False)  # Saving the Files Again



### Schema to Parse different CSV:

In [29]:
class InputSchema(pw.Schema):
  SystemCodeNumber:str
  Capacity : int
  Occupancy : int
  VehicleType : str
  TrafficConditionNearby : str
  QueueLength  : int
  IsSpecialDay : int
  Timestamp :  str   # Currently parsed it as 'str' , afterward I would parsed it to Datetime
  color : str  # Optional Feature to show the Special Day in Plot for better Visualisation
  vehicle_type_weight : float
  traffic_level : float

* Reading the 14 CSV files as the Streaming Data :

In [30]:
tables = {}
for i in range(1, 15):
    filename = f"/content/parking_lot_{i}.csv"
    table_name = f"t{i}"
    tables[table_name] = pw.demo.replay_csv(     # Making a dictionary as tables in which 14 parking lots are streamed as values
        f"{filename}",
        schema=InputSchema,
        input_rate=1000   # 1000 rows read per second (decent speed)
    )

* Parsing the Date & Time using DateTimeNaive(dt)

In [31]:
fmt = "%Y-%m-%d %H:%M:%S"  # format of current Timestamp Column
for i in range(1,15):
  tables[f't{i}'] = tables[f't{i}'].with_columns(
   TimeStamp = tables[f't{i}'].Timestamp.dt.strptime(fmt)   #parsed to Datetime
  )
for i in range(1,15):
  tables[f't{i}'] = tables[f't{i}'].with_columns(
    day = tables[f't{i}'].Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")  #made a new feature as the Single Day
  )

## Model Creation

### ***Using the Demand Formula*** :  
Demand of the Parking lot depend on various Factor such as follow:
1. **Length of Queue**:

  Longer the Queue , Higher is the Demand  
2. **Traffic In the Neighbourhood**:

   More is the Traffic in Neighbour , Less is the ability for Cars to come , Less is the Demand
3. **Type of Vehicle that want to park**:

   Larger the Vehicle , More is the Ability of Driver to Pay
4. **Utilisation Ratio** : Occupancy / Capacity

   Higher the Ratio , More is the Demand
5. **Is Day a Holiday / Special Day or a Normal Day:**

   If the Day is special , Demand will be higher Before the Day and on the Day .

* *alpha , beta , gamma, delta, epsilon  are the Coefficient that manages how the demand changes with each of the* *Factors* :

* *lambda_ is coefficient that manages the normalisation of demand between the range of Max_demand and Min_demand*

In [32]:
base_price = 10 # given
# carefully manually tuned to get plot with minimum fluctuation
alpha = 1.5
beta = 0.8
gamma = 0.5
delta = 1.5
epsilon = 1.2
lambda_ = 0.9
min_demand = 0   # Minimum demand is obv 0
max_demand = 10  # Maximum demand ,let's say 10

* *Confirming if all the Columns are Parsed Correctly , and None of it has 'typing.Any' Datatype*

In [33]:
tables['t1'].typehints()

mappingproxy({'SystemCodeNumber': str,
              'Capacity': int,
              'Occupancy': int,
              'VehicleType': str,
              'TrafficConditionNearby': str,
              'QueueLength': int,
              'IsSpecialDay': int,
              'Timestamp': str,
              'color': str,
              'vehicle_type_weight': float,
              'traffic_level': float,
              'TimeStamp': pathway.internals.datetime_types.DateTimeNaive,
              'day': str})

### Windowing the Dataset

* Windowing the Datasets by 30 Minutes Duration and Aggregate by windowby().reduce() Method

In [34]:
from datetime import timedelta   # to assign the duration of window with timedelta


windowed_table = {}  # Made the dictionary and making the changes
for i in range(1,15):
  table_name = f't{i}'
  windowed_table[table_name] = (
      tables[f't{i}'].windowby(
          pw.this.TimeStamp,   #use this columns to divide window
          instance=pw.this.day, #use this a Partioning key
          window= pw.temporal.tumbling(duration =timedelta(minutes=30)), #Made the tumbling window of 30 minutes
          behavior=pw.temporal.exactly_once_behavior()   # so that each row is computed exactly once
      )
      .reduce(
          time_start = pw.this._pw_window_start,   # made this columns for debugging
          time_end=pw.this._pw_window_end,         # act as the timestamp for plotting
          occ_max=pw.reducers.max(pw.this.Occupancy),  # selecting max occupancy in window
          cap=pw.reducers.max(pw.this.Capacity),  # selecting max capacity in window
          color = pw.reducers.earliest(pw.this.color), # used this features for Special Day plotting
          traffic_level = pw.reducers.earliest(pw.this.traffic_level), # selecting the first traffic_level
          vehicle_type_weight = pw.reducers.earliest(pw.this.vehicle_type_weight),#selecting first vehicle weight
          IsSpecialDay = pw.reducers.earliest(pw.this.IsSpecialDay),  # used for computing demand
          queue_length = pw.reducers.earliest(pw.this.QueueLength), # used for computing demand
      ).with_columns(
        demand =  (   # Calculating the Demand Function
          alpha * (pw.this.occ_max/pw.this.cap) +
          beta * pw.this.queue_length -
          gamma * pw.this.traffic_level +
          delta * pw.this.IsSpecialDay +
          epsilon * pw.this.vehicle_type_weight
       )
      )
      .with_columns(  # Normalizing demand as to in the range (0,10)
          normalized_demand = ((pw.this.demand - min_demand) / (max_demand - min_demand))
      )
      .with_columns( #calculating Price
          temp_price = (base_price * (1 + (lambda_ * pw.this.normalized_demand)))
      )
      .with_columns( #Normalizing Price as to in the range(5,20)
          price = pw.if_else(
            pw.this.temp_price < base_price * 0.5,
            base_price * 0.5,
            pw.if_else(
              pw.this.temp_price > base_price * 2,
              base_price * 2,
              pw.this.temp_price
            )
         )
      )
  )


/tmp/ipython-input-24-3355532624.py:14: UserWarning: earliest reducer uses processing time to choose elements while windowby uses data time to assign entries to windows. Maybe it is not the behavior you want. To choose elements according to their data time, you may use min reducer.
  .reduce(
/tmp/ipython-input-34-3355532624.py:14: UserWarning: earliest reducer uses processing time to choose elements while windowby uses data time to assign entries to windows. Maybe it is not the behavior you want. To choose elements according to their data time, you may use min reducer.
  .reduce(


## Creating the Interactive Dashboard

* Making the Dashboard of Different Tabs using the Bokeh and Panel Libraries

In [36]:
import panel as pn
pn.extension()   #Enabling Panel's widgets and layout features
def price_plotter(source):  #defining the Price Plotter function
    fig = bokeh.plotting.figure(  #creating a fig by bokeh
        height=400,
        width=800,
        title=" Dynamic Parking Lot Price",
        tools = 'hover,pan,zoom_in,zoom_out,wheel_zoom,save,reset',  #manually setting the tools required
        x_axis_label  = 'Time',
        y_axis_label = 'Price in Dollar',
        x_axis_type="datetime" )
    fig.line("time_end", "price", source=source, line_width=2, color="navy")
    fig.circle("time_end", "price", source=source, size=6, color="color")
    return fig
figures = {}
for i in range(1,15):
  figures[f'fig{i}']= windowed_table[f't{i}'].plot(price_plotter, sorting_col="time_end")  #creating the figures of different parking lot

In [37]:
tabs = pn.Tabs( # Making the Dashboard with multiple tabs for different Parking Lot
    *[(f"Lot {i}", figures[f"fig{i}"]) for i in range(1, 15)]
)
tabs.servable()

Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

## Creating the DataFlow Using Pathway

In [38]:
pw.run()   # Starting the Pipeline

Output()